# USING PRE TRAINED MODEL


In [104]:
!pip install transformers torch sentencepiece
import pandas as pd


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [105]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

# Pretrained QA model
model_name = "distilbert/distilbert-base-cased-distilled-squad"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

In [106]:
def extractive_qa(context, question):
    inputs = tokenizer(
        question,
        context,
        return_tensors="pt",
        truncation=True,
        max_length=512
    )
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    start = torch.argmax(outputs.start_logits)
    end = torch.argmax(outputs.end_logits)

    answer_ids = inputs["input_ids"][0][start : end + 1]
    answer = tokenizer.decode(answer_ids, skip_special_tokens=True)

    return answer


In [108]:
context = "Once upon a time, a man died due to a heart attack."
question = "How did the man die?"
extractive_qa(context, question)


'heart attack'

**The pre-trained model successfully extracts relevant information from the given context**

# Hybrid RAG (Extractive + Abstractive).

In [109]:
#Libraries import
!pip install faiss-cpu transformers sentence-transformers accelerate
import faiss 
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from nltk.tokenize import sent_tokenize

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [115]:
##This function splits a long paragraph into multiple smaller overlapping chunks using a sliding window technique.
def chunk_text(text, chunk_size=300, overlap=90):
    words = text.split()
    chunks = []
    start = 0
    while start < len(words):
        end = start + chunk_size
        chunk = " ".join(words[start:end])
        chunks.append(chunk)
        start += (chunk_size - overlap)
    return chunks


##This code converts the paragraph into semantic sentence embeddings using MiniLM, then stores them in a FAISS L2 index for fast similarity-based retrieval.
model=SentenceTransformer('all-MiniLM-L6-v2')
paragraph=input("paste your paragraph:\n").lower().strip()
chunks=chunk_text(paragraph)
paragraph = " ".join(chunks)
sentence=sent_tokenize(paragraph)
encoded_sent=model.encode(sentence)
dimension=encoded_sent.shape[1]
index=faiss.IndexFlatL2(dimension)
index.add(encoded_sent)


##The retrive() function performs semantic search by converting the query into embeddings and retrieving the top-k most similar sentences from the FAISS index.
def retrive(q,k=5):
    q=model.encode([q])
    dist,idx=index.search(q,k)
    context=[sentence[i] for i in idx[0]]
    return ". ".join(context)


#This initializes a text-generation pipeline using the Qwen 1.5B Instruct model, which will be used to generate answers based on the retrieved context.
qa = pipeline(
    "text-generation",
    model="Qwen/Qwen2.5-1.5B-Instruct",
    device=0
)

##This loop continuously takes user questions, retrieves the most relevant context using FAISS, generates an answer using the Qwen model, and prints only the final cleaned answer. The loop stops when the user types ‘exit’
while True:
    print("Enter 'exit' to quit\n")
    question = input("Ask your question: ").lower().strip()
    if question == 'exit':
        print("Bye bye")
        break
    context = retrive(question)
    prompt = f"""Answer the question based only on the given context.

Context:
{context}

Question:
{question}

Answer:"""
    print("Generating answer...")
    raw_output = qa(prompt, max_new_tokens=200)[0]["generated_text"]
    final_answer = raw_output.split("Answer:")[-1].split("\n")[0].strip()
    print("\nAnswer:", final_answer)

paste your paragraph:
 The rapid evolution of technology over the last few decades has transformed nearly every aspect of human life. From communication to transportation, medicine to education, digital systems now shape how societies function and how individuals interact with the world around them. In the early 2000s, the internet was still developing, but its potential was already evident. People used slow dial-up connections to access simple websites, send emails, or join online forums. Today, the internet has become a global ecosystem of information and services, supporting billions of devices and enabling real-time communication across continents. This transformation has brought immense opportunities but also new challenges that require continuous adaptation.  One of the most influential developments has been the rise of smartphones. When the first touchscreen devices appeared, they were considered luxurious gadgets intended for professionals or tech enthusiasts. Over time, they b

Device set to use cuda:0


Enter 'exit' to quit



Ask your question:  How has the rise of smartphones impacted daily human life according to the passage?


Generating answer...

Answer: The passage states that smartphones have become "personal assistants, navigation systems, entertainment centers, and gateways to social media." It also mentions that they are used for communication and information gathering. Therefore, the impact of smartphones on daily human life is significant because they enable people to stay connected with others through various means such as text messages, emails, instant messaging apps, video calls, and social networking sites. They also provide a platform for accessing news, weather updates, traffic reports, sports scores, and other relevant information in real-time. Additionally, smartphones facilitate shopping online, banking transactions, making appointments, managing schedules, playing games, watching videos, listening to music, reading books, and learning new skills. Overall, smartphones have revolutionized how we live our lives by providing us with access to an endless amount of information and allowing us to

...

Generating answer...

Answer: The ethical concerns associated with the use of AI include issues related to data security, bias in decision-making processes, and the potential for job displacement. Policymakers and technologists need to work together to address these challenges by ensuring responsible and transparent usage of AI technologies.
Enter 'exit' to quit



Ask your question:  How has technology influenced modern work culture and remote work?


Generating answer...

Answer: Technology has significantly impacted modern work culture by enabling remote work, transforming how employees interact with each other and collaborate. It has also facilitated communication and collaboration regardless of geographical location, leading to more flexible working arrangements and increased productivity. The rise of remote work has been driven by advancements in digital tools and platforms that make it easier for teams to communicate, share information, and complete tasks without being physically present in the same space. This shift towards remote work not only enhances flexibility but also opens up new opportunities for talent from diverse locations, fostering a global workforce. Furthermore, technological innovations have revolutionized project management, data analysis, and decision-making processes within organizations, streamlining workflows and improving efficiency. In summary, technology has transformed the landscape of work, making it

Ask your question:  exit


Bye bye


In [119]:
##The rapid evolution of technology over the last few decades has transformed nearly every aspect of human life. From communication to transportation, medicine to education, digital systems now shape how societies function and how individuals interact with the world around them. In the early 2000s, the internet was still developing, but its potential was already evident. People used slow dial-up connections to access simple websites, send emails, or join online forums. Today, the internet has become a global ecosystem of information and services, supporting billions of devices and enabling real-time communication across continents. This transformation has brought immense opportunities but also new challenges that require continuous adaptation.  One of the most influential developments has been the rise of smartphones. When the first touchscreen devices appeared, they were considered luxurious gadgets intended for professionals or tech enthusiasts. Over time, they became essential tools for nearly everyone. Smartphones now serve as personal assistants, navigation systems, entertainment centers, and gateways to social media. They allow people to carry the world’s knowledge in their pockets, making information more accessible than at any point in history. However, this convenience has also contributed to concerns about privacy, distraction, and digital addiction. Many researchers argue that the constant use of mobile devices can affect attention spans and interpersonal relationships. As a result, society is still learning how to balance the benefits of connectivity with the need for mental well-being.  Artificial intelligence has become another defining force of the modern era. Early AI systems were limited to rule-based programs that could perform only simple tasks. But with the rise of machine learning and deep neural networks, AI has grown into a powerful tool capable of analyzing vast amounts of data and making complex predictions. Industries such as healthcare, finance, manufacturing, and entertainment now rely on AI to improve efficiency and accuracy. For example, machine learning models help doctors detect diseases earlier, support financial institutions in identifying fraudulent transactions, and assist autonomous vehicles in navigating safely through traffic. Despite these advancements, AI raises important ethical questions about data security, bias, and potential job displacement. Policymakers and technologists must collaborate to ensure that AI is used responsibly and transparently.  The spread of digital technologies has also changed how people learn. Traditional classrooms, once limited by physical space and printed textbooks, have evolved into hybrid learning environments where students can access multimedia resources, join virtual discussions, and participate in online courses taught by experts from around the world. During global events such as the COVID-19 pandemic, online education became a necessity rather than a choice. Although remote learning helped maintain academic continuity, it also exposed inequalities in access to devices and stable internet connections. Students from underserved communities often struggled to participate fully, highlighting the need for more equitable digital infrastructure. As education continues to evolve, schools must adopt teaching methods that encourage critical thinking, creativity, and collaboration, skills essential for thriving in a technology-driven future.  In addition to learning, the nature of work has changed dramatically. Automation and digital tools have eliminated many repetitive tasks while creating new opportunities in fields like software development, cybersecurity, and data analytics. Remote work, once a rare privilege, has become widely accepted across industries. Employees can now collaborate virtually, attend meetings from home, and manage teams across different time zones. This flexibility has improved work–life balance for many people, but it has also blurred the boundaries between personal and professional time. Organizations must create policies that support employee well-being, encourage productivity, and protect against burnout. The future of work will likely involve a combination of remote and on-site roles, requiring companies to rethink their use of physical office spaces and digital tools.  Environmental sustainability is another major topic influenced by technology. Innovations in renewable energy, electric vehicles, and smart agriculture offer solutions to reduce carbon emissions and conserve natural resources. Solar panels, wind turbines, and energy-efficient appliances have become more affordable and widespread. Urban planners use data analytics to design smart cities that optimize transportation, reduce waste, and improve air quality. At the same time, the production and disposal of electronic devices contribute to pollution and e-waste, posing environmental challenges that must be addressed. Governments and industries must work together to promote recycling, responsible manufacturing, and sustainable consumption. Technology alone cannot solve climate change, but it can play a crucial role in guiding global efforts.  Social interactions have also been transformed by digital platforms. Social media allows people to share ideas, build communities, and stay connected with friends and family across great distances. However, these platforms can also spread misinformation, encourage unhealthy comparisons, and amplify social divisions. Algorithms designed to keep users engaged sometimes create echo chambers that reinforce existing beliefs rather than promoting constructive dialogue. As digital citizens, individuals must learn to evaluate information critically and use online platforms responsibly. At the same time, technology companies must develop systems that prioritize user safety, transparency, and ethical content moderation.  Despite the many benefits of technology, there is growing concern about privacy and data protection. Every time people use a smartphone, browse the internet, or engage with digital services, they generate data that can be collected, analyzed, or shared. While data helps companies improve user experiences and develop innovative products, it can also be misused for surveillance, targeted advertising, or identity theft. Regulations such as the General Data Protection Regulation (GDPR) aim to protect individuals by giving them more control over their personal information. However, enforcing these regulations across global digital platforms remains challenging. Users must become more aware of their digital rights, and companies must adopt stronger safeguards to protect sensitive data.  Looking ahead, the future of technology promises both excitement and uncertainty. Emerging fields such as quantum computing, biotechnology, and advanced robotics have the potential to reshape industries once again. Quantum computers could solve problems currently impossible for classical machines, such as designing new materials or optimizing complex systems. Biotechnology may lead to breakthroughs in medicine, agriculture, and environmental restoration. Advanced robotics could transform manufacturing, healthcare, and even household tasks. Yet, each of these technologies raises ethical, social, and economic questions. It is essential for governments, scientists, and communities to work together to ensure that technological progress benefits all of humanity rather than a select few.  Ultimately, technology is a tool—powerful, transformative, and constantly evolving. Its impact depends on how societies choose to use it. By fostering innovation, promoting ethical practices, and prioritizing human well-being, the global community can ensure that the digital age contributes to a more inclusive, sustainable, and prosperous future for everyone.

In [120]:
#Q.1= How has the rise of smartphones impacted daily human life according to the passage?

#Q.2= Why did online education become essential during the COVID-19 pandemic?

#Q.3= What are some ethical concerns associated with the use of artificial intelligence?

#Q.4=How has technology influenced modern work culture and remote work?

#Q.5=What environmental challenges are created by the production and disposal of electronic devices, and how can technology help address them?